In [13]:
import pymc as pm
import numpy as np
import pandas as pd
import pymc.sampling_jax
import xarray as xr
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import sklearn.preprocessing as P
import sklearn.compose as C
from sklearn.pipeline import Pipeline



### 5.1

In [21]:
df = pd.read_csv("../Data/WaffleDivorce.csv", sep=";")
df.head()

,Location,Loc,Population,MedianAgeMarriage,Marriage,Marriage SE,Divorce,Divorce SE,WaffleHouses,South,Slaves1860,Population1860,PropSlaves1860
0,Alabama,AL,4.78,25.3,20.2,1.27,12.7,0.79,128,1,435080,964201,0.45
1,Alaska,AK,0.71,25.2,26.0,2.93,12.5,2.05,0,0,0,0,0.00
2,Arizona,AZ,6.33,25.8,20.3,0.98,10.8,0.74,18,0,0,0,0.00
3,Arkansas,AR,2.92,24.3,26.4,1.70,13.5,1.22,41,1,111115,435450,0.26
4,California,CA,37.25,26.8,19.1,0.39,8.0,0.24,0,0,0,379994,0.00


Scale predictors of interest:

In [22]:
ct = C.ColumnTransformer(
    [
        ("norm1", P.StandardScaler(), ["MedianAgeMarriage"]),
        ("norm2", P.StandardScaler(), ["Divorce"])
    ]
)
df.loc[:, ["A", "D"]] = ct.fit_transform(df)

### 5.2

In [23]:
df["MedianAgeMarriage"].std()

1.2436303013880823

### 5.3

In [24]:
with pm.Model() as m_5_1:
    alpha = pm.Normal("alpha", mu=0, sigma=0.2)
    beta_A = pm.Normal("beta_A", mu=0, sigma=0.5)
    sigma = pm.Exponential("sigma", lam=1)
    mu = pm.Deterministic("mu", alpha + beta_A * df["A"])
    D = pm.Normal("D", mu=mu, sigma=sigma, observed=df["D"])

### 5.4

In [ ]:
with m_5_1:
    smp_prior_predictive = pm.sample_prior_predictive()

In [27]:
smp_prior_predictive["prior"].to_dataframe().reset_index

sigma        mu    beta_A     alpha
chain draw mu_dim_0                                        
0     0    0         0.143205 -0.499599  0.553859 -0.160391
           1         0.143205 -0.544587  0.553859 -0.160391
           2         0.143205 -0.274660  0.553859 -0.160391
           3         0.143205 -0.949477  0.553859 -0.160391
           4         0.143205  0.175218  0.553859 -0.160391
...                       ...       ...       ...       ...
      499  45        1.499593  0.276308  0.463528  0.146037
           46        1.499593  0.088055  0.463528  0.146037
           47        1.499593 -0.250800  0.463528  0.146037
           48        1.499593  0.238657  0.463528  0.146037
           49        1.499593 -0.552005  0.463528  0.146037

[25000 rows x 4 columns]